In [ ]:
from flask import Flask, jsonify
import psycopg2
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

def get_db_connection():
    try:
        conn = psycopg2.connect(
            dbname="project",
            user="postgres",
            password="admin",
            host="localhost"
        )
        return conn
    except psycopg2.Error as e:
        app.logger.error(f"Error connecting to database: {e}")
        return None

@app.route('/landslides/surface', methods=['GET'])
def get_landslides():
    conn = get_db_connection()
    if conn is None:
        return jsonify({"error": "Database connection error"}), 500
    
    try:
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute(" SELECT ST_AsText(geometry) AS geometry_wkt, ar_kmq , nome, ar_fr_p2, ar_fr_p1, ar_fr_p3p4, ar_frp3p4p FROM dataset ;")
        landslides = cursor.fetchall()
        cursor.close()
        conn.close()
        return jsonify(landslides)
    except psycopg2.Error as e:
        app.logger.error(f"Error executing SQL query: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=5000, use_reloader=False)


@app.route('/landslides/surface', methods=['GET'])

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jun/2024 12:12:08] "GET /landslides/surface HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 12:12:08] "GET /landslides/surface HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 12:12:12] "GET /landslides/surface HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 12:12:12] "GET /landslides/surface HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 12:12:14] "GET /landslides/surface HTTP/1.1" 200 -
